In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 51.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=b61f5e05253008ac20e094d5f1239b64d791b3a4b5e13e8bb3e55fb21fc1b44b
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
#import SparkSession
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('log_reg').getOrCreate()

In [3]:
#read the dataset
# import pandas as pd
df=spark.read.csv('Log_Reg_dataset.csv',inferSchema=True,header=True)

In [4]:
from pyspark.sql.functions import *


In [5]:
#check the shape of the data 
print((df.count(),len(df.columns)))

(20000, 6)


In [6]:
#printSchema
df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Repeat_Visitor: integer (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Web_pages_viewed: integer (nullable = true)
 |-- Status: integer (nullable = true)



In [7]:
#number of columns in dataset
df.columns

['Country', 'Age', 'Repeat_Visitor', 'Platform', 'Web_pages_viewed', 'Status']

In [8]:
#view the dataset
df.show(5)

+---------+---+--------------+--------+----------------+------+
|  Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|
+---------+---+--------------+--------+----------------+------+
|    India| 41|             1|   Yahoo|              21|     1|
|   Brazil| 28|             1|   Yahoo|               5|     0|
|   Brazil| 40|             0|  Google|               3|     0|
|Indonesia| 31|             1|    Bing|              15|     1|
| Malaysia| 32|             0|  Google|              15|     1|
+---------+---+--------------+--------+----------------+------+
only showing top 5 rows



In [9]:
#Exploratory Data Analysis
df.describe().show()


+-------+--------+-----------------+-----------------+--------+-----------------+------------------+
|summary| Country|              Age|   Repeat_Visitor|Platform| Web_pages_viewed|            Status|
+-------+--------+-----------------+-----------------+--------+-----------------+------------------+
|  count|   20000|            20000|            20000|   20000|            20000|             20000|
|   mean|    null|         28.53955|           0.5029|    null|           9.5533|               0.5|
| stddev|    null|7.888912950773227|0.500004090187782|    null|6.073903499824976|0.5000125004687693|
|    min|  Brazil|               17|                0|    Bing|                1|                 0|
|    max|Malaysia|              111|                1|   Yahoo|               29|                 1|
+-------+--------+-----------------+-----------------+--------+-----------------+------------------+



In [10]:
df.groupBy('Country').count().show()

+---------+-----+
|  Country|count|
+---------+-----+
| Malaysia| 1218|
|    India| 4018|
|Indonesia|12178|
|   Brazil| 2586|
+---------+-----+



In [11]:
df.groupBy('Platform').count().show()

+--------+-----+
|Platform|count|
+--------+-----+
|   Yahoo| 9859|
|    Bing| 4360|
|  Google| 5781|
+--------+-----+



In [12]:
df.groupBy('Status').count().show()

+------+-----+
|Status|count|
+------+-----+
|     1|10000|
|     0|10000|
+------+-----+



In [13]:
df.groupBy('Country').mean().show()

+---------+------------------+-------------------+---------------------+--------------------+
|  Country|          avg(Age)|avg(Repeat_Visitor)|avg(Web_pages_viewed)|         avg(Status)|
+---------+------------------+-------------------+---------------------+--------------------+
| Malaysia|27.792282430213465| 0.5730706075533661|   11.192118226600986|  0.6568144499178982|
|    India|27.976854156296664| 0.5433051269288203|   10.727227476356397|  0.6212045793927327|
|Indonesia| 28.43159796354081| 0.5207751683363442|    9.985711939563148|  0.5422893742814913|
|   Brazil|30.274168600154677|  0.322892498066512|    4.921113689095128|0.038669760247486466|
+---------+------------------+-------------------+---------------------+--------------------+



In [14]:
df.groupBy('Platform').mean().show()

+--------+------------------+-------------------+---------------------+------------------+
|Platform|          avg(Age)|avg(Repeat_Visitor)|avg(Web_pages_viewed)|       avg(Status)|
+--------+------------------+-------------------+---------------------+------------------+
|   Yahoo|28.569226087838523| 0.5094837204584644|    9.599655137437875|0.5071508266558474|
|    Bing| 28.68394495412844| 0.4720183486238532|    9.114908256880733|0.4559633027522936|
|  Google|28.380038055699707| 0.5149628092025601|    9.804878048780488|0.5210171250648676|
+--------+------------------+-------------------+---------------------+------------------+



In [15]:
df.groupBy('Status').mean().show()

+------+--------+-------------------+---------------------+-----------+
|Status|avg(Age)|avg(Repeat_Visitor)|avg(Web_pages_viewed)|avg(Status)|
+------+--------+-------------------+---------------------+-----------+
|     1| 26.5435|             0.7019|              14.5617|        1.0|
|     0| 30.5356|             0.3039|               4.5449|        0.0|
+------+--------+-------------------+---------------------+-----------+



In [16]:
#converting categorical data to numerical form

In [17]:
#import required libraries

from pyspark.ml.feature import StringIndexer


In [18]:
#Indexing 

In [19]:
Platform_indexer = StringIndexer(inputCol="Platform", outputCol="Platform_Num").fit(df)
df = Platform_indexer.transform(df)

In [20]:
df.show(3,False)

+-------+---+--------------+--------+----------------+------+------------+
|Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|Platform_Num|
+-------+---+--------------+--------+----------------+------+------------+
|India  |41 |1             |Yahoo   |21              |1     |0.0         |
|Brazil |28 |1             |Yahoo   |5               |0     |0.0         |
|Brazil |40 |0             |Google  |3               |0     |1.0         |
+-------+---+--------------+--------+----------------+------+------------+
only showing top 3 rows



In [21]:
from pyspark.ml.feature import OneHotEncoder

In [22]:
#one hot encoding
# temp = df
Platform_encoder = OneHotEncoder(inputCol="Platform_Num", outputCol="Platform_Vector")
df1 = Platform_encoder.fit(df)
df = df1.transform(df)

In [23]:
df.show(3,False)

+-------+---+--------------+--------+----------------+------+------------+---------------+
|Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|Platform_Num|Platform_Vector|
+-------+---+--------------+--------+----------------+------+------------+---------------+
|India  |41 |1             |Yahoo   |21              |1     |0.0         |(2,[0],[1.0])  |
|Brazil |28 |1             |Yahoo   |5               |0     |0.0         |(2,[0],[1.0])  |
|Brazil |40 |0             |Google  |3               |0     |1.0         |(2,[1],[1.0])  |
+-------+---+--------------+--------+----------------+------+------------+---------------+
only showing top 3 rows



In [24]:
df.groupBy('Platform').count().orderBy('count',ascending=False).show(5,False)

+--------+-----+
|Platform|count|
+--------+-----+
|Yahoo   |9859 |
|Google  |5781 |
|Bing    |4360 |
+--------+-----+



In [25]:
df.groupBy('Platform_Num').count().orderBy('count',ascending=False).show(5,False)

+------------+-----+
|Platform_Num|count|
+------------+-----+
|0.0         |9859 |
|1.0         |5781 |
|2.0         |4360 |
+------------+-----+



In [26]:
df.groupBy('Platform_Vector').count().orderBy('count',ascending=False).show(5,False)

+---------------+-----+
|Platform_Vector|count|
+---------------+-----+
|(2,[0],[1.0])  |9859 |
|(2,[1],[1.0])  |5781 |
|(2,[],[])      |4360 |
+---------------+-----+



In [27]:
country_indexer = StringIndexer(inputCol="Country", outputCol="Country_Num").fit(df)
df = country_indexer.transform(df)

In [28]:
df.select(['Country','Country_Num']).show(3,False)

+-------+-----------+
|Country|Country_Num|
+-------+-----------+
|India  |1.0        |
|Brazil |2.0        |
|Brazil |2.0        |
+-------+-----------+
only showing top 3 rows



In [29]:
df

DataFrame[Country: string, Age: int, Repeat_Visitor: int, Platform: string, Web_pages_viewed: int, Status: int, Platform_Num: double, Platform_Vector: vector, Country_Num: double]

In [30]:
#one hot encoding
country_encoder = OneHotEncoder(inputCol="Country_Num", outputCol="Country_Vector")
df2 = country_encoder.fit(df)
df = df2.transform(df)

In [31]:
df.select(['Country','country_Num','Country_Vector']).show(3,False)

+-------+-----------+--------------+
|Country|country_Num|Country_Vector|
+-------+-----------+--------------+
|India  |1.0        |(3,[1],[1.0]) |
|Brazil |2.0        |(3,[2],[1.0]) |
|Brazil |2.0        |(3,[2],[1.0]) |
+-------+-----------+--------------+
only showing top 3 rows



In [32]:
df.groupBy('Country').count().orderBy('count',ascending=False).show(5,False)

+---------+-----+
|Country  |count|
+---------+-----+
|Indonesia|12178|
|India    |4018 |
|Brazil   |2586 |
|Malaysia |1218 |
+---------+-----+



In [33]:
df.groupBy('Country_Num').count().orderBy('count',ascending=False).show(5,False)

+-----------+-----+
|Country_Num|count|
+-----------+-----+
|0.0        |12178|
|1.0        |4018 |
|2.0        |2586 |
|3.0        |1218 |
+-----------+-----+



In [34]:
df.groupBy('Country_Vector').count().orderBy('count',ascending=False).show(5,False)

+--------------+-----+
|Country_Vector|count|
+--------------+-----+
|(3,[0],[1.0]) |12178|
|(3,[1],[1.0]) |4018 |
|(3,[2],[1.0]) |2586 |
|(3,[],[])     |1218 |
+--------------+-----+



In [35]:
from pyspark.ml.feature import VectorAssembler

In [36]:
df_assembler = VectorAssembler(inputCols=['Age', 'Repeat_Visitor','Web_pages_viewed'], outputCol="features")
df = df_assembler.transform(df)

In [37]:
df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Repeat_Visitor: integer (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Web_pages_viewed: integer (nullable = true)
 |-- Status: integer (nullable = true)
 |-- Platform_Num: double (nullable = false)
 |-- Platform_Vector: vector (nullable = true)
 |-- Country_Num: double (nullable = false)
 |-- Country_Vector: vector (nullable = true)
 |-- features: vector (nullable = true)



In [38]:
df.select(['features','Status']).show(10,False)

+---------------+------+
|features       |Status|
+---------------+------+
|[41.0,1.0,21.0]|1     |
|[28.0,1.0,5.0] |0     |
|[40.0,0.0,3.0] |0     |
|[31.0,1.0,15.0]|1     |
|[32.0,0.0,15.0]|1     |
|[32.0,0.0,3.0] |0     |
|[32.0,0.0,6.0] |0     |
|[27.0,0.0,9.0] |0     |
|[32.0,0.0,2.0] |0     |
|[31.0,1.0,16.0]|1     |
+---------------+------+
only showing top 10 rows



In [39]:
#select data for building model
model_df=df.select(['features','Status'])

In [40]:
from pyspark.ml.classification import LogisticRegression

In [41]:
#split the data 
training_df,test_df=model_df.randomSplit([0.75,0.25])

In [42]:
training_df.count()

15042

In [43]:
training_df.groupBy('Status').count().show()

+------+-----+
|Status|count|
+------+-----+
|     1| 7542|
|     0| 7500|
+------+-----+



In [44]:
test_df.count()

4958

In [45]:
test_df.groupBy('Status').count().show()

+------+-----+
|Status|count|
+------+-----+
|     1| 2458|
|     0| 2500|
+------+-----+



In [46]:
log_reg=LogisticRegression(labelCol='Status').fit(training_df)

In [47]:
#Training Results

In [48]:
train_results=log_reg.evaluate(training_df).predictions

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [49]:
train_results.filter(train_results['Status']==1).filter(train_results['prediction']==1).select(['Status','prediction','probability']).show(10,False)

+------+----------+----------------------------------------+
|Status|prediction|probability                             |
+------+----------+----------------------------------------+
|1     |1.0       |[0.35802508920717824,0.6419749107928218]|
|1     |1.0       |[0.35802508920717824,0.6419749107928218]|
|1     |1.0       |[0.35802508920717824,0.6419749107928218]|
|1     |1.0       |[0.35802508920717824,0.6419749107928218]|
|1     |1.0       |[0.35802508920717824,0.6419749107928218]|
|1     |1.0       |[0.35802508920717824,0.6419749107928218]|
|1     |1.0       |[0.2098715193203292,0.7901284806796708] |
|1     |1.0       |[0.2098715193203292,0.7901284806796708] |
|1     |1.0       |[0.2098715193203292,0.7901284806796708] |
|1     |1.0       |[0.2098715193203292,0.7901284806796708] |
+------+----------+----------------------------------------+
only showing top 10 rows



Probability at 0 index is for 0 class and probabilty as 1 index is for 1 class

In [50]:
correct_preds=train_results.filter(train_results['Status']==1).filter(train_results['prediction']==1).count()


In [51]:
training_df.filter(training_df['Status']==1).count()

7542

In [52]:
#accuracy on training dataset 
float(correct_preds)/(training_df.filter(training_df['Status']==1).count())

0.9277380005303633

In [53]:
#Test Set results

In [54]:
results=log_reg.evaluate(test_df).predictions

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [55]:
results.select(['Status','prediction']).show(10,False)

+------+----------+
|Status|prediction|
+------+----------+
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
+------+----------+
only showing top 10 rows



In [56]:
results.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Status: integer (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [57]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [58]:
#confusion matrix
true_postives = results[(results.Status == 1) & (results.prediction == 1)].count()
true_negatives = results[(results.Status == 0) & (results.prediction == 0)].count()
false_positives = results[(results.Status == 0) & (results.prediction == 1)].count()
false_negatives = results[(results.Status == 1) & (results.prediction == 0)].count()

In [59]:
print (true_postives)
print (true_negatives)
print (false_positives)
print (false_negatives)
print(true_postives+true_negatives+false_positives+false_negatives)
print (results.count())

2311
2325
175
147
4958
4958


In [60]:
recall = float(true_postives)/(true_postives + false_negatives)
print(recall)

0.9401952807160293


In [61]:
precision = float(true_postives) / (true_postives + false_positives)
print(precision)

0.9296057924376508


In [62]:
accuracy=float((true_postives+true_negatives) /(results.count()))
print(accuracy)

0.9350544574425171
